In [1]:
import torch.nn as nn
import torch
input = torch.randn(3,22,15000)
input.shape

torch.Size([3, 22, 15000])

In [2]:
nn.Conv1d(in_channels=22,out_channels=32,kernel_size=2,stride=2,padding=0)


Conv1d(22, 32, kernel_size=(2,), stride=(2,))

In [3]:
class Block(nn.Module):
    def __init__(self,inplace):
        super().__init__()
        self.conv1= nn.Conv1d(in_channels=inplace,out_channels=32,kernel_size=2,stride=2,padding=0)
        self.conv2= nn.Conv1d(in_channels=inplace,out_channels=32,kernel_size=4,stride=2,padding=1)
        self.conv3= nn.Conv1d(in_channels=inplace,out_channels=32,kernel_size=8,stride=2,padding=3)
        self.relu=nn.ReLU()
        
    def forward(self,x):
        print(x.shape)
        x1=self.conv1(x)
        x2=self.conv2(x)
        x3=self.conv3(x)
        print(x1.shape,x2.shape,x3.shape)
        x=torch.cat([x1,x2,x3],dim=1)
        return x

In [4]:
block = Block(22)
out1=block(input)
out1.shape

torch.Size([3, 22, 15000])
torch.Size([3, 32, 7500]) torch.Size([3, 32, 7500]) torch.Size([3, 32, 7500])


torch.Size([3, 96, 7500])

In [5]:
block = Block(96)
out2=block(out1)
out2.shape

torch.Size([3, 96, 7500])
torch.Size([3, 32, 3750]) torch.Size([3, 32, 3750]) torch.Size([3, 32, 3750])


torch.Size([3, 96, 3750])

In [6]:
block = Block(96)
out3=block(out2)
out3.shape

torch.Size([3, 96, 3750])
torch.Size([3, 32, 1875]) torch.Size([3, 32, 1875]) torch.Size([3, 32, 1875])


torch.Size([3, 96, 1875])

In [7]:
gru = nn.GRU(input_size=96,hidden_size=32,batch_first=True)
x=out3.permute(0,2,1)
print(x.shape)
output,hn= gru(x) 
print(output.shape,hn.shape)

torch.Size([3, 1875, 96])
torch.Size([3, 1875, 32]) torch.Size([1, 3, 32])


In [8]:
gru1 = nn.GRU(input_size=96,hidden_size=32,batch_first=True)
gru2 = nn.GRU(input_size=32,hidden_size=32,batch_first=True)
gru_out1,_=gru1(x)
gru_out2,_=gru2(gru_out1)

gru_out=torch.cat([gru_out1,gru_out2],dim=2)
gru_out.shape

torch.Size([3, 1875, 64])

In [9]:
gru3=nn.GRU(input_size=64,hidden_size=32,batch_first=True)
gru_out3,_=gru3(gru_out)
gru_out3.shape

torch.Size([3, 1875, 32])

In [10]:
gru_out = torch.cat([gru_out1,gru_out2,gru_out3],dim=2)
gru_out.shape

torch.Size([3, 1875, 96])

In [11]:
gru4=nn.GRU(input_size=96,hidden_size=32,batch_first=True)
gru_out4,_=gru4(gru_out)
gru_out4.shape

torch.Size([3, 1875, 32])

In [12]:
linear=nn.Linear(1875,1)
linear_out=linear(gru_out.permute(0,2,1))
linear_out.shape

torch.Size([3, 96, 1])

In [13]:
gru4=nn.GRU(input_size=96,hidden_size=32,batch_first=True)
gru_out4,_=gru4(linear_out.permute(0,2,1))
gru_out4.shape

torch.Size([3, 1, 32])

In [80]:
class ChronoNet(nn.Module):
    def __init__(self,channel):
        super().__init__()
        self.block1=Block(channel)
        self.block2=Block(96)
        self.block3=Block(96)
        self.gru1=nn.GRU(input_size=96,hidden_size=32,batch_first=True)
        self.gru2=nn.GRU(input_size=32,hidden_size=32,batch_first=True)
        self.gru3=nn.GRU(input_size=64,hidden_size=32,batch_first=True)
        self.gru4=nn.GRU(input_size=96,hidden_size=32,batch_first=True)
        self.gru_linear=nn.Linear(64,1)
        self.flatten=nn.Flatten()
        self.fc1=nn.Linear(32,1)
        self.relu=nn.ReLU()
        
    def forward(self,x):
        x=self.block1(x)
        x=self.block2(x)
        x=self.block3(x)
        x=x.permute(0,2,1)
        gru_out1,_=self.gru1(x)
        gru_out2,_=self.gru2(gru_out1)
        gru_out=torch.cat([gru_out1,gru_out2],dim=2)
        gru_out3,_=self.gru3(gru_out)
        gru_out=torch.cat([gru_out1,gru_out2,gru_out3],dim=2)
        #print('gru_out',gru_out.shape)
        linear_out=self.relu(self.gru_linear(gru_out.permute(0,2,1)))
        gru_out4,_=self.gru4(linear_out.permute(0,2,1))
        x=self.flatten(gru_out4)
        x=self.fc1(x)
        return x
     

input=torch.randn(3,14,512)
input.shape
model=ChronoNet(14)
out=model(input)

torch.Size([3, 14, 512])
torch.Size([3, 32, 256]) torch.Size([3, 32, 256]) torch.Size([3, 32, 256])
torch.Size([3, 96, 256])
torch.Size([3, 32, 128]) torch.Size([3, 32, 128]) torch.Size([3, 32, 128])
torch.Size([3, 96, 128])
torch.Size([3, 32, 64]) torch.Size([3, 32, 64]) torch.Size([3, 32, 64])


In [54]:
out.shape

torch.Size([3, 1])

In [55]:
out=model(input)


torch.Size([3, 14, 512])
torch.Size([3, 32, 256]) torch.Size([3, 32, 256]) torch.Size([3, 32, 256])
torch.Size([3, 96, 256])
torch.Size([3, 32, 128]) torch.Size([3, 32, 128]) torch.Size([3, 32, 128])
torch.Size([3, 96, 128])
torch.Size([3, 32, 64]) torch.Size([3, 32, 64]) torch.Size([3, 32, 64])


In [56]:
out.shape

torch.Size([3, 1])

In [57]:
from glob import glob
import os
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [58]:
all_files_path=glob('EEG 14S - 14H/*.edf')
print(len(all_files_path))

28


In [59]:
healthy_file_path=[i for i in all_files_path if  'h' in i.split('\\')[1]]
patient_file_path=[i for i in all_files_path if  's' in i.split('\\')[1]]

In [60]:
def read_data(file_path):
    datax=mne.io.read_raw_edf(file_path,preload=True)
    datax.set_eeg_reference()
    datax.filter(l_freq=1,h_freq=45)
    epochs=mne.make_fixed_length_epochs(datax,duration=25,overlap=0)
    epochs=epochs.get_data()
    return epochs #trials,channel,length

In [61]:
data=read_data(healthy_file_path[0])

Extracting EDF parameters from C:\Users\hpw\OneDrive\Documents\pytorch course\EEG 14S - 14H\h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 s)

Not setting metadata


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished


37 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 37 events and 6250 original time points ...
0 bad epochs dropped


In [62]:
data.shape

(37, 19, 6250)

In [63]:
control_epochs_array=[read_data(subject) for subject in healthy_file_path]
patients_epochs_array=[read_data(subject) for subject in patient_file_path]

Extracting EDF parameters from C:\Users\hpw\OneDrive\Documents\pytorch course\EEG 14S - 14H\h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 s)

Not setting metadata
37 matching events found
No baseline correction applied
0 projec

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished


0 bad epochs dropped
Extracting EDF parameters from C:\Users\hpw\OneDrive\Documents\pytorch course\EEG 14S - 14H\h02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 227499  =      0.000 ...   909.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 s)

Not setting metadata


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished


36 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 36 events and 6250 original time points ...
0 bad epochs dropped
Extracting EDF parameters from C:\Users\hpw\OneDrive\Documents\pytorch course\EEG 14S - 14H\h03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 227499  =      0.000 ...   909.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transit

...

[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.1s finished


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
86 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 86 events and 6250 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.3s finished


In [64]:
control_epochs_labels=[len(i)*[0] for i in control_epochs_array]
patients_epochs_labels=[len(i)*[1] for i in patients_epochs_array]
print(len(control_epochs_labels),len(patients_epochs_labels))

14 14


In [65]:
data_list=control_epochs_array+patients_epochs_array
label_list=control_epochs_labels+patients_epochs_labels
print(len(data_list),len(label_list))

28 28


In [66]:
groups_list=[[i]*len(j) for i, j in enumerate(data_list)]
print(len(groups_list))

28


In [67]:
from sklearn.model_selection import GroupKFold,LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gkf=GroupKFold()
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.preprocessing import StandardScaler
#https://stackoverflow.com/questions/50125844/how-to-standard-scale-a-3d-matrix
class StandardScaler3D(BaseEstimator,TransformerMixin):
    #batch, sequence, channels
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self,X,y=None):
        self.scaler.fit(X.reshape(-1, X.shape[2]))
        return self

    def transform(self,X):
        return self.scaler.transform(X.reshape( -1,X.shape[2])).reshape(X.shape)

In [68]:

import numpy as np
data_array=np.concatenate(data_list)
label_array=np.concatenate(label_list)
group_array=np.concatenate(groups_list)
data_array=np.moveaxis(data_array,1,2)

print(data_array.shape,label_array.shape,group_array.shape)
     

(1142, 6250, 19) (1142,) (1142,)


In [69]:
accuracy=[]
for train_index, val_index in gkf.split(data_array, label_array, groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler3D()
    train_features=scaler.fit_transform(train_features)
    val_features=scaler.transform(val_features)
    train_features=np.moveaxis(train_features,1,2)
    val_features=np.moveaxis(val_features,1,2)
    break
    accuracy.append(model_train(train_features,val_features,train_labels,val_labels))

    

In [70]:
train_features = torch.Tensor(train_features)
val_features = torch.Tensor(val_features)
train_labels = torch.Tensor(train_labels)
val_labels = torch.Tensor(val_labels)

In [71]:
len(val_features),len(val_labels)

(231, 231)

In [72]:

train_labels.shape

torch.Size([911])

In [73]:
from pytorch_lightning import LightningModule,Trainer
import torchmetrics
from torch.utils.data import TensorDataset,DataLoader

In [74]:
class ChronoModel(LightningModule):
    def __init__(self):
        super(ChronoModel,self).__init__()
        self.model=ChronoNet(19)
        self.lr=1e-3
        self.bs=12
        self.worker=2
        self.acc=torchmetrics.Accuracy(task='binary')
        self.creterion=nn.BCEWithLogitsLoss()
        
    def forward(self,x):
        x=self.model(x)
        return x
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(),lr=self.lr)
    
    def train_dataloader(self):
        dataset=TensorDataset(train_features,train_labels)
        dataloader=DataLoader(dataset,batch_size=self.bs,num_workers=self.worker,shuffle=True)
        return dataloader
    
    def training_step(self,batch,batch_idx):
        signal,label=batch
        out=self(signal.float())
        loss=self.creterion(out.flatten(),label.float().flatten())
        acc=self.acc(out.flatten(),label.long().flatten())
        return {'loss':loss,'acc':acc}
    
    
    def trained_epoch_end(self,outputs):
        acc=torch.stack([x['acc'] for x in outputs]).mean().detach().cpu().numpy()
        loss=torch.stack([x['loss'] for x in outputs]).mean().detach().cpu().numpy()
        print('train acc loss',acc,loss)
        
    def val_dataloader(self):
        dataset=TensorDataset(val_features,val_labels)
        dataloader=DataLoader(dataset,batch_size=self.bs,num_workers=self.worker,shuffle=True)
        return dataloader
    
    def validation_step(self,batch,batch_idx):
        signal,label=batch
        out=self(signal.float())
        loss=self.creterion(out.flatten(),label.float().flatten())
        acc=self.acc(out.flatten(),label.long().flatten())
        return {'loss':loss,'acc':acc}
        
    def on_validation_epoch_end(self,outputs):
        acc=torch.stack([x['acc'] for x in outputs]).mean().detach().cpu().numpy()
        loss=torch.stack([x['loss'] for x in outputs]).mean().detach().cpu().numpy()
        print('val acc loss',acc,loss)
        
    
        
        
    

In [75]:
model=ChronoModel()

In [76]:
trainer=Trainer(max_epochs=1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [77]:

trainer.fit(model)


  | Name      | Type              | Params
------------------------------------------------
0 | model     | ChronoNet         | 135 K 
1 | acc       | BinaryAccuracy    | 0     
2 | creterion | BCEWithLogitsLoss | 0     
------------------------------------------------
135 K     Trainable params
0         Non-trainable params
135 K     Total params
0.542     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\hpw\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:480: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
C:\Users\hpw\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


torch.Size([12, 19, 6250])
torch.Size([12, 32, 3125]) torch.Size([12, 32, 3125]) torch.Size([12, 32, 3125])
torch.Size([12, 96, 3125])
torch.Size([12, 32, 1562]) torch.Size([12, 32, 1562]) torch.Size([12, 32, 1562])
torch.Size([12, 96, 1562])
torch.Size([12, 32, 781]) torch.Size([12, 32, 781]) torch.Size([12, 32, 781])


RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [12, 781] but got: [12, 64].